# **Table des matières**

[**Imports et fonctions**](#Imports-et-fonctions)

[**Exploration et préparation des données**](#Exploration-et-préparation)
   - [Cinq datasets](#Cinq-datasets)
   - [population](#population)
   - [sous_nutrition](#sous_nutrition)
   - [aliments (vegetaux + animaux)](#aliments-(vegetaux-+-animaux))
   - [cereales](#cereales)
   - [Résumé](#Résumé)
   - [Hypothèses retenues](#Hypothèses-retenues)

[**Questions 1 à 14**](#Questions-1-à-14) 
   - [Question 1 : Nombre d'humains sur la planète](#Q1-:-Nombre-d'humains-sur-la-planète)
   - [Question 2 : Redondances](#Q2-:-Redondances)
   - [Question 3 : Dispo. alimentaire (calories, protéines)](#Q3-:-Dispo.-alimentaire-(calories,-protéines))
   - [Question 4 : Ratio énergie/poids](#Q4-:-Ratio-énergie/poids)
   - [Question 5 : Aliments les plus caloriques et protéiques](#Q5-:-Aliments-les-plus-caloriques-et-protéiques)
   - [Question 6 : Dispo. intérieure mondiale des végétaux](#Q6-:-Dispo.-intérieure-mondiale-des-végétaux-(calories))
   - [Question 7 : Potentiel alimentaire des végétaux](#Q7-:-Potentiel-alimentaire-des-végétaux)
   - [Question 8 : Potentiel alimentaire des végétaux (destinés aux animaux pertes)](#Q8-:-Potentiel-alimentaire-des-végétaux-(destinés-aux-animaux-+-pertes))
   - [Question 9 : Potentiel alimentaire de la dispo. mondiale](#Q9-:-Potentiel-alimentaire-de-la-dispo.-mondiale)
   - [Question 10 : Pourcentage de la sous-nutrition mondiale](#Q10-:-Pourcentage-de-la-sous-nutrition-mondiale)
   - [Question 11 : Céréales](#Q11-:-Céréales)
   - [Question 12 : Sous-nutrition](#Q12-:-Sous-nutrition)
   - [Question 13 : USA](#Q13-:-USA)
   - [Question 14 : Thaïlande](#Q14-:-Thaïlande)

[**Questions 15 à 20**](#Questions-15-à-20)
   - [Question 15 : Table population](#Q15-:-Table-population)
   - [Question 16 : Table dispo_alim](#Q16-:-Table-dispo_alim)
   - [Question 17 : Table equilibre_prod](#Q17-:-Table-equilibre_prod)
   - [Question 18 : Table sous_nutrition](#Q18-:-Table-sous_nutrition)
   - [Question 19 : Requêtes SQL](#Q19-:-Requêtes-SQL)
   - [Question 20 : Autres utilisations des produits](#Q20-:-Autres-utilisations-des-produits)

[**Analyses supplémentaires**](#Analyses-supplémentaires)
   - [Nombre de personnes sous-alimentées](#Nombre-de-personnes-sous-alimentées)
   - [Décès dus à la faim](#Décès-dus-à-la-faim)
   - [Évolutions possibles pour 2050](#Évolutions-possibles-pour-2050)
   - [Causes de la sous-alimentation](#Causes-de-la-sous-alimentation)      
   
[**Sources**](#Sources)

# **Imports et fonctions**

In [1]:
# Import des librairies
import numpy as np
import pandas as pd

# Affichage des valeurs : séparer les milliers, et réduire le nombre de décimales
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Import des fichiers et création des dataframes
url = 'sources/fr_'
population =       pd.read_csv(url + 'population' + '.csv')
sous_nutrition =   pd.read_csv(url + 'sousalimentation' + '.csv')    
animaux =          pd.read_csv(url + 'animaux' + '.csv')
cereales =         pd.read_csv(url + 'cereales' + '.csv')    
vegetaux =         pd.read_csv(url + 'vegetaux' + '.csv')

In [3]:
# Liste des dataframes
listDf = [population, sous_nutrition, animaux, cereales, vegetaux]
listDfStr = ['population', 'sous_nutrition', 'animaux', 'cereales', 'vegetaux']

In [4]:
# Fonction d'exploration des datasets
def exploration(df):
    
    # Noms des colonnes, null-count, types d'objets 
    print(df.info())
    
    print('\n')
    for i in range(0, len(df.columns)):
        
        # Nom de la colonne
        print(df.columns[i], ':')
        
        # Nombre de Nan 
        print(sum(df.iloc[:, i].isna()), 'Nan')
        
        # Nombre de zéros
        print(df.shape[0] - np.count_nonzero(df.iloc[:, i]), 'zéros')
        
        # Nombre de valeurs uniques 
        print(len(df.iloc[:, i].unique()), 'valeurs uniques :')
        print(df.iloc[:, i].unique())
        
        # .describe()
        print(df.iloc[:, i].describe())
        
        print('\n')

In [5]:
# Fonction de recherche de clé primaire (une seule colonne)

def pk(df,col): # Ne pas oublier les guillemets en déclarant le paramètre col

    x = 0
    df = df.reset_index()
    
    for i in range(0,len(df)):
    
        # Valeur à rechercher
        searched = df.loc[i][col]

        # Rechercher cette valeur dans toute la df, et supprimer les duplicates pour vérifier combien de lignes on obtient
        s = df[df[col]==searched].drop_duplicates()
        
        # Si on obtient une seule ligne, c'est a priori une clé primaire
        if len(s) == 1:
            x += 1
    
    # Si pour chaque ligne de la df, la requêtes a ajouté 1 à x, alors x = la longueur de la dataframe
    if x == len(df):
        print('\'', col, '\' est bien une clé primaire.')
    else:
        print('\'', col, '\' n\'est pas une clé primaire.')

# **Exploration et préparation**

## Cinq datasets
- Les datasets contiennent les mêmes nombres et intitulés de colonnes, à l'exception de sous_nutrition, qui compte en plus la colonne 'Note'

In [6]:
population.sample(1)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
173,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,251,Zambie,511,Population totale,2501,Population,2013,2013,1000 personnes,14539,NaN,Donnée officielle


In [7]:
sous_nutrition.sample(1)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
136,FS,Données de la sécurité alimentaire,21,Brésil,6132,Valeur,210011,Nombre de personnes sous-alimentées (millions)...,20132015,2013-2015,millions,NaN,NR,Non rapportée,nan


In [8]:
animaux.sample(1)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
4289,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,17,Bermudes,5154,Autres utilisations (non alimentaire),2781,Huiles de Poissons,2013,2013,Milliers de tonnes,0.00,S,Données standardisées


In [9]:
vegetaux.sample(1)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
25861,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,48,Costa Rica,684,Disponibilité de matière grasse en quantité (g...,2615,Bananes,2013,2013,g/personne/jour,0.00,Fc,Donnée calculée


In [10]:
cereales.sample(1)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
197,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,44,Colombie,5511,Production,2511,Blé,2013,2013,Milliers de tonnes,8,S,Données standardisées


In [11]:
# Renommer les différentes colonnes des datasets
for i in range(0, len(listDf)):
    listDf[i].rename(columns={
        'Code zone' : 'code_pays',
        'Zone' : 'pays',
        'Année' : 'annee',
        'Produit' : 'produit',
        'Code Produit' : 'code_produit'
    }, inplace=True)    

In [12]:
# Exploration des 5 datasets
for i in range(0, len(listDf)):
    print(
        listDfStr[i], ':',
        
        # Dimensions
        listDf[i].shape[1], 'colonnes, ',
        listDf[i].shape[0], 'lignes \n \n',
        
        # Noms des colonnes 
        listDf[i].columns, '\n \n'
    )

population : 14 colonnes,  175 lignes 
 
 Index(['Code Domaine', 'Domaine', 'code_pays', 'pays', 'Code Élément',
       'Élément', 'code_produit', 'produit', 'Code année', 'annee', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole'],
      dtype='object') 
 

sous_nutrition : 15 colonnes,  1020 lignes 
 
 Index(['Code Domaine', 'Domaine', 'code_pays', 'pays', 'Code Élément',
       'Élément', 'code_produit', 'produit', 'Code année', 'annee', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole', 'Note'],
      dtype='object') 
 

animaux : 14 colonnes,  37166 lignes 
 
 Index(['Code Domaine', 'Domaine', 'code_pays', 'pays', 'Code Élément',
       'Élément', 'code_produit', 'produit', 'Code année', 'annee', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole'],
      dtype='object') 
 

cereales : 14 colonnes,  891 lignes 
 
 Index(['Code Domaine', 'Domaine', 'code_pays', 'pays', 'Code Élément',
       'Élément', 'code_produit', 'produit', 'Code année', 'anne

## population
- A l'exception de 'Code zone', 'Zone', 'Valeur', 'Symbole' et 'Description du Symbole', toutes les colonnes contiennent une seule valeur
- Ces colonnes ne sont pas des clés, elles sont inutiles à l'exception de 'Année' (voir table [sous_nutrition](#sous_nutrition))
- Les colonnes 'Symbole' et 'Description du Symbole' ne contiennent que 2 valeurs indiquant la présence d'un agrégat (voir [question 1](#Question-1))
- Les colonnes 'Zone' et 'Code zone' contiennent les pays et leur code (175 au total, soit 19 de mois que le [nombre d'états membres de la FAO](http://www.fao.org/about/who-we-are/fr/#:~:text=La%20FAO%20est%20une%20organisation,membre%2C%20l'Union%20europ%C3%A9enne.))

In [13]:
exploration(population)    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Code Domaine            175 non-null    object
 1   Domaine                 175 non-null    object
 2   code_pays               175 non-null    int64 
 3   pays                    175 non-null    object
 4   Code Élément            175 non-null    int64 
 5   Élément                 175 non-null    object
 6   code_produit            175 non-null    int64 
 7   produit                 175 non-null    object
 8   Code année              175 non-null    int64 
 9   annee                   175 non-null    int64 
 10  Unité                   175 non-null    object
 11  Valeur                  175 non-null    int64 
 12  Symbole                 1 non-null      object
 13  Description du Symbole  175 non-null    object
dtypes: int64(6), object(8)
memory usage: 19.3+ KB
None


Code 

In [14]:
# Renommer la colonne 'Valeur'
population.rename(columns={'Valeur' : 'population'}, inplace=True)

# Projection des colonnes contenant plus d'une valeur (excepté 'annee')
# Conserver temporairement 'Description du Symbole', en prévision de la question 1
population = population[['code_pays','pays','population','annee','Description du Symbole']]

# Conversion de la population en habitants
population['population'] = population['population'] * 1000

# Affichage
population.sample(3)

<ipython-input-14-281ff81a35e9>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population['population'] = population['population'] * 1000


,code_pays,pays,population,annee,Description du Symbole
103,134,Malte,429000,2013,Donnée officielle
8,9,Argentine,41446000,2013,Donnée officielle
35,128,Chine - RAS de Macao,566000,2013,Donnée officielle


## sous_nutrition
- Seules les colonnes 'Code zone', 'Zone', 'Code année', 'Année', 'Valeur', 'Symbole et Description du Symbole' contiennent plus d'une valeur
- Contrairement à population, la colonne 'Unité' référence en millions la colonne 'Valeur' (mais la colonne valeur contient 40% de nulls)
- La colonne 'Valeur' contient des valeurs non numériques ('<0.1')
- Cette même colonne 'Valeur' n'est pas de type int, et contient des strings
- La colonne 'Note' est vide (seule colonne non commune aux autres tables)
- Contrairement à population, animaux et vegetaux, on compte ici plus de pays (204) que le FAO ne compte d'états membres
- Contrairement aux autres tables, la colonne année va de 2012-2014 à 2016-2018

In [15]:
exploration(sous_nutrition)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            1020 non-null   object 
 1   Domaine                 1020 non-null   object 
 2   code_pays               1020 non-null   int64  
 3   pays                    1020 non-null   object 
 4   Code Élément            1020 non-null   int64  
 5   Élément                 1020 non-null   object 
 6   code_produit            1020 non-null   int64  
 7   produit                 1020 non-null   object 
 8   Code année              1020 non-null   int64  
 9   annee                   1020 non-null   object 
 10  Unité                   1020 non-null   object 
 11  Valeur                  605 non-null    object 
 12  Symbole                 1020 non-null   object 
 13  Description du Symbole  1020 non-null   object 
 14  Note                    0 non-null      

In [16]:
# Renommer la colonne 'Valeur'
sous_nutrition.rename(columns={
    'Valeur' : 'nb_personnes'
}, inplace=True)

# Projection
sous_nutrition = sous_nutrition[['code_pays','pays','nb_personnes','annee']]

# Remplacer les '<0.1' (par 0)
sous_nutrition['nb_personnes'].replace('<0.1', '0', inplace=True)

# La colonne nb_personnes ne contient que des strings
print('Type des valeurs de la colonne \'nb_personnes\' :', type(sous_nutrition['nb_personnes'].loc[5]), '\n')

# Convertir la colonne nb_personnes en float
sous_nutrition['nb_personnes'] = sous_nutrition['nb_personnes'].astype(float)

# Conversion de nb_personnes en habitants
sous_nutrition['nb_personnes'] = sous_nutrition['nb_personnes'] * 1000000

# Affichage
print(sous_nutrition.sample(3))

Type des valeurs de la colonne 'nb_personnes' : <class 'str'> 

     code_pays                    pays  nb_personnes      annee
588        136              Mauritanie    400,000.00  2015-2017
786         56  République dominicaine  1,300,000.00  2013-2015
43         194         Arabie saoudite  2,000,000.00  2015-2017


c:\users\guifo\appdata\local\programs\python\python38\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-16-38c88a28aea6>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sous_nutrition['nb_personnes'] = sous_nutrition['nb_personnes'].astype(float)
<ipython-input-16-38c88a28aea6>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

## aliments (vegetaux + animaux)
- Ces 2 tables comportent les mêmes structures et valeurs, à l'exception des colonnes référençant les produits et valeurs

In [17]:
exploration(animaux)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37166 entries, 0 to 37165
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            37166 non-null  object 
 1   Domaine                 37166 non-null  object 
 2   code_pays               37166 non-null  int64  
 3   pays                    37166 non-null  object 
 4   Code Élément            37166 non-null  int64  
 5   Élément                 37166 non-null  object 
 6   code_produit            37166 non-null  int64  
 7   produit                 37166 non-null  object 
 8   Code année              37166 non-null  int64  
 9   annee                   37166 non-null  int64  
 10  Unité                   37166 non-null  object 
 11  Valeur                  37166 non-null  float64
 12  Symbole                 37166 non-null  object 
 13  Description du Symbole  37166 non-null  object 
dtypes: float64(1), int64(5), object(8)
mem

In [18]:
exploration(vegetaux)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104871 entries, 0 to 104870
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Code Domaine            104871 non-null  object 
 1   Domaine                 104871 non-null  object 
 2   code_pays               104871 non-null  int64  
 3   pays                    104871 non-null  object 
 4   Code Élément            104871 non-null  int64  
 5   Élément                 104871 non-null  object 
 6   code_produit            104871 non-null  int64  
 7   produit                 104871 non-null  object 
 8   Code année              104871 non-null  int64  
 9   annee                   104871 non-null  int64  
 10  Unité                   104871 non-null  object 
 11  Valeur                  104871 non-null  float64
 12  Symbole                 104871 non-null  object 
 13  Description du Symbole  104871 non-null  object 
dtypes: float64(1), int64

0 Nan
32006 zéros
6249 valeurs uniques :
[5169.   1173.   -350.   ... 3880.     28.7    20.63]
count   104,871.00
mean        321.27
std       6,395.04
min     -39,863.00
25%           0.00
50%           1.00
75%          17.00
max     739,267.00
Name: Valeur, dtype: float64


Symbole :
0 Nan
0 zéros
3 valeurs uniques :
['S' 'Fc' 'A']
count     104871
unique         3
top            S
freq       67182
Name: Symbole, dtype: object


Description du Symbole :
0 Nan
0 zéros
3 valeurs uniques :
['Données standardisées' 'Donnée calculée'
 'Agrégat, peut inclure des données officielles, semi-officielles, estimées ou calculées']
count                    104871
unique                        3
top       Données standardisées
freq                      67182
Name: Description du Symbole, dtype: object




In [19]:
# Valeurs inférieures à 0 ??? 
print('Animaux : \n', animaux['Valeur'].describe(), '\n')
print(vegetaux['Valeur'].describe())

Animaux : 
 count    37,166.00
mean        148.29
std       1,933.73
min      -1,368.00
25%           0.00
50%           1.00
75%          11.00
max     135,600.00
Name: Valeur, dtype: float64 

count   104,871.00
mean        321.27
std       6,395.04
min     -39,863.00
25%           0.00
50%           1.00
75%          17.00
max     739,267.00
Name: Valeur, dtype: float64


In [20]:
# Ajout d'une colonne 'Origine' dans animaux et vegetaux
animaux['origine'] = 'Animale'
vegetaux['origine'] = 'Végétale'

# Union de Végétaux et Animaux
aliments = pd.concat([vegetaux, animaux])

# Projection
aliments = aliments[[
    'pays',
    'code_pays',
    'produit',
    'code_produit',
    'origine',
    'Élément',
    'Valeur',
    'annee'
]]

# Table pivot sur aliments
aliments = aliments.pivot_table('Valeur', index=['code_pays','pays','produit','code_produit','annee','origine'], columns='Élément')

# Reset index
aliments.reset_index(inplace=True)

# Renommer les colonnes
aliments.rename(columns={
    'Aliments pour animaux' : 'alim_ani',
    'Autres utilisations (non alimentaire)' : 'autres_utilisations',
    'Semences' : 'semences',
    'Pertes' : 'pertes',
    'Nourriture' : 'nourriture',
    'Traitement' : 'transfo', # ?
    'Disponibilité intérieure' : 'dispo_int',
    'Disponibilité alimentaire (Kcal/personne/jour)' : 'dispo_alim_kcal_p_j',
    'Disponibilité de protéines en quantité (g/personne/jour)' : 'dispo_prot',
    'Disponibilité de matière grasse en quantité (g/personne/jour)' : 'dispo_mat_gr'
}, inplace=True)

# Joindre population et aliments
aliments = pd.merge(aliments, population[['pays', 'population']], on="pays")

# Remplacer les Nan
aliments.replace(np.nan, 0, inplace=True)

# Affichage
print('Résultat : ', aliments.shape[1], 'colonnes, ', aliments.shape[0], 'lignes')
aliments.sample(3)

Résultat :  22 colonnes,  15702 lignes


,code_pays,pays,produit,code_produit,annee,origine,alim_ani,autres_utilisations,dispo_alim_kcal_p_j,Disponibilité alimentaire en quantité (kg/personne/an),...,dispo_int,Exportations - Quantité,Importations - Quantité,nourriture,pertes,Production,semences,transfo,Variation de stock,population
6273,102,Iran (République islamique d'),Maïs,2514,2013,Végétale,"5,369.00",0.00,21.00,2.37,...,"5,866.00",4.00,"4,007.00",184.00,293.00,"1,852.00",20.00,0.00,11.00,77447000
7901,121,Liban,Pois,2547,2013,Végétale,0.00,0.00,7.00,0.79,...,4.00,0.00,1.00,4.00,0.00,4.00,0.00,0.00,0.00,4822000
9273,144,Mozambique,Huile de Coco,2578,2013,Végétale,0.00,0.00,26.00,1.06,...,27.00,2.00,0.00,27.00,0.00,29.00,0.00,0.00,0.00,25834000


In [21]:
exploration(aliments)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15702 entries, 0 to 15701
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   code_pays                                               15702 non-null  int64  
 1   pays                                                    15702 non-null  object 
 2   produit                                                 15702 non-null  object 
 3   code_produit                                            15702 non-null  int64  
 4   annee                                                   15702 non-null  int64  
 5   origine                                                 15702 non-null  object 
 6   alim_ani                                                15702 non-null  float64
 7   autres_utilisations                                     15702 non-null  float64
 8   dispo_alim_kcal_p_j                 

count   15,702.00
mean        94.05
std        918.75
min       -201.00
25%          0.00
50%          1.00
75%         16.00
max     65,564.00
Name: Importations - Quantité, dtype: float64


nourriture :
0 Nan
5616 zéros
1440 valeurs uniques :
[1.4000e+01 1.0000e+00 0.0000e+00 ... 5.4689e+04 1.9449e+04 2.1290e+03]
count    15,702.00
mean        391.49
std       5,677.91
min        -246.00
25%           0.00
50%           3.00
75%          41.00
max     430,046.00
Name: nourriture, dtype: float64


pertes :
0 Nan
12478 zéros
449 valeurs uniques :
[0.0000e+00 3.2000e+01 1.0000e+00 1.9000e+01 2.7000e+01 7.0000e+00
 1.5000e+01 8.0000e+00 9.6000e+01 1.2000e+01 1.1000e+01 2.0000e+00
 7.7500e+02 1.7000e+01 5.8000e+01 1.2200e+02 3.0000e+00 3.1000e+01
 5.0000e+00 5.2000e+01 4.0000e+00 9.0000e+00 2.4000e+01 4.4000e+01
 8.7000e+01 2.2000e+01 2.3000e+01 6.0000e+00 6.7200e+02 3.4000e+01
 6.1000e+01 1.5600e+02 3.9800e+02 2.2500e+02 1.0000e+01 6.7000e+01
 1.2100e+02 2.8000e+01 1.7730e+03 7.8000e+01 

## cereales
Cette table ne contient que des redondances de la table vegetaux, elle ne servira qu'à catégoriser certains de ces produits végétaux.

In [22]:
exploration(cereales)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Code Domaine            891 non-null    object
 1   Domaine                 891 non-null    object
 2   code_pays               891 non-null    int64 
 3   pays                    891 non-null    object
 4   Code Élément            891 non-null    int64 
 5   Élément                 891 non-null    object
 6   code_produit            891 non-null    int64 
 7   produit                 891 non-null    object
 8   Code année              891 non-null    int64 
 9   annee                   891 non-null    int64 
 10  Unité                   891 non-null    object
 11  Valeur                  891 non-null    int64 
 12  Symbole                 891 non-null    object
 13  Description du Symbole  891 non-null    object
dtypes: int64(6), object(8)
memory usage: 97.6+ KB
None


Code 

## *Résumé*
- L'ensemble des tables couvre uniquement l'année 2013, excepté sous_nutrition, qui contient des données de 2012 à 2015 : on a donc gardé la colonne 'annee' pour population et sous_nutrition
- Présence de Nan et de zéros :
    - **population** : aucun
    - **sous_nutrition** : 40% de Nan dans la colonne 'nb_habitants'
    - **aliments** (avant pivot) : 32% de zéros dans la colonne 'Valeur'
    - **aliments** (après pivot) : de 25% à 90% de zéros dans toutes les colonnes (excepté 'population')

## Hypothèses retenues

In [23]:
# Besoin calorique annuel / personne
# = 2300 kilocalories * 365 jours
besoin_kcal = 2000 * 365

# Besoin protéique annuel / personne
# = 0.9 gramme de protéines * 62kg (poids moyen d'un humain) * 365 jours / 1000 (conversion en kg) 
besoin_kgprot = .9 * 62 * 365 / 1000

# Seuil à partir duquel la FAO déclare un pays en sous-nutrition : 5%
# Source : https://fr.wikipedia.org/wiki/Liste_des_pays_par_taux_de_malnutrition
seuil_sousnutrition = 5

# **Questions 1 à 14**

## Q1 : Nombre d'humains sur la planète
Calculez le nombre total d’humains sur la planète. Critiquez votre résultat. En cas d’anomalie, analysez et effectuer les corrections nécessaires. Donnez le résultat de votre calcul pour l'année 2013.

In [24]:
population_monde = population['population'].sum()
population_monde_wikipedia = 7210582000


print(
    "Population mondiale : ⚠️", round((population_monde) / 10**9, 2), "milliards d'habitants \n",
    '(chiffre incohérent comparé aux', round((population_monde_wikipedia / 10**9), 2), 'milliards estimés pour 2013) \n\n',
    'Source : https://en.wikipedia.org/wiki/World_population'
)

Population mondiale : ⚠️ 8.41 milliards d'habitants 
 (chiffre incohérent comparé aux 7.21 milliards estimés pour 2013) 

 Source : https://en.wikipedia.org/wiki/World_population


In [25]:
# Les données de la Chine ont été agrégées dans le code pays 351, et donc comptabilisées 2 fois
population[population['pays'].str.contains('Chine')]

,code_pays,pays,population,annee,Description du Symbole
33,351,Chine,1416667000,2013,"Agrégat, peut inclure des données officielles,..."
34,96,Chine - RAS de Hong-Kong,7204000,2013,Donnée officielle
35,128,Chine - RAS de Macao,566000,2013,Donnée officielle
36,41,"Chine, continentale",1385567000,2013,Donnée officielle
37,214,"Chine, Taiwan Province de",23330000,2013,Donnée officielle


In [26]:
# Somme des données agrégées
somme_agregat = population[population['code_pays'].isin([96,128,41,214])]['population'].sum()

print(
    'Valeur du code 351 :', '{:,}'.format(int(population[population['code_pays']==351]['population'].sum())), '\n',
    'Somme des codes 96, 128, 41 et 214 :', '{:,}'.format(somme_agregat)
)

Valeur du code 351 : 1,416,667,000 
 Somme des codes 96, 128, 41 et 214 : 1,416,667,000


In [27]:
# Fonction de suppression de l'agrégat (code pays 351)
def suppression_agregat(df):
    df.drop(df[df['code_pays']==351].index, inplace=True)
    
# Suppression de l'agrégat dans les dataframes
suppression_agregat(population)
suppression_agregat(sous_nutrition)
suppression_agregat(aliments)

# Drop de la colonne Description dans population, maintenant inutile
population.drop(columns=['Description du Symbole'], inplace=True)

print(population[population['pays'].str.contains('Chine')])

    code_pays                       pays  population  annee
34         96   Chine - RAS de Hong-Kong     7204000   2013
35        128       Chine - RAS de Macao      566000   2013
36         41        Chine, continentale  1385567000   2013
37        214  Chine, Taiwan Province de    23330000   2013


c:\users\guifo\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\users\guifo\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\users\guifo\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [28]:
# Population mondiale corrigée
population_monde = population['population'].sum()

print(
    "Population mondiale (corrigée) : 👌", round((population_monde) / 10**9, 2), "milliards d'habitants \n",
    '(', round(population_monde / population_monde_wikipedia * 100), '% de l\'estimation de Wikipedia)' 
)

Population mondiale (corrigée) : 👌 7.0 milliards d'habitants 
 ( 97 % de l'estimation de Wikipedia)


## Q2 : Redondances
Identifiez ces redondances, en donnant votre réponse sous forme de formule mathématique (pas besoin de coder ici :soleil: ). C'est une équation à 3 termes de type (a_1 + a2 + [...] = b_1 + b_2 + [...] = c_1 + c_2 + [...]) ) faisant intervenir chacune des 11 quantités données ci dessus. Illustrez cette équation avec l'exemple du blé en France.

In [29]:
# Formule
# Disponibilité intérieure = Production + Importations + Variation - Exportations = Nourriture + Aliments animaux + Semences + Pertes + Traitement + Autres utilisations

# Exemple du blé en France
bf = aliments[(aliments['pays']=='France') & (aliments['produit']=='Blé')].reset_index()

print(
    bf.loc[0, 'dispo_int'],
    '=',
    bf.loc[0, 'Production']
    + bf.loc[0, 'Importations - Quantité']
    + bf.loc[0, 'Variation de stock']
    - bf.loc[0, 'Exportations - Quantité'],
    '=',
    bf.loc[0, 'nourriture']
    + bf.loc[0, 'alim_ani']
    + bf.loc[0, 'semences']
    + bf.loc[0, 'pertes']
    + bf.loc[0, 'transfo']
    + bf.loc[0, 'autres_utilisations']
)      

20298.0 = 20298.0 = 20298.0


## Q3 : Dispo. alimentaire (calories, protéines)
Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines. Vous ferez cela à partir de ces informations : Population de chaque pays; Disponibilité alimentaire donnée pour chaque produit et pour chaque pays en kcal/personne/jour, Disponibilité alimentaire en protéines donnée pour chaque produit et pour chaque pays en g/personne/jour.

In [30]:
# Création des colonnes Disponibilité en kcal/an et kgprot/an
aliments['dispo_kcal_an'] = aliments['dispo_alim_kcal_p_j'] * aliments['population'] * 365
aliments['dispo_kgprot_an'] = aliments['dispo_prot'] / 1000 * aliments['population'] * 365

print(len(aliments))
aliments.head(3)

15605


,code_pays,pays,produit,code_produit,annee,origine,alim_ani,autres_utilisations,dispo_alim_kcal_p_j,Disponibilité alimentaire en quantité (kg/personne/an),...,Importations - Quantité,nourriture,pertes,Production,semences,transfo,Variation de stock,population,dispo_kcal_an,dispo_kgprot_an
0,1,Arménie,Abats Comestible,2736,2013,Animale,0.00,3.00,14.00,4.59,...,2.00,14.00,0.00,15.00,0.00,0.00,0.00,2977000,"15,212,470,000.00","2,444,861.25"
1,1,Arménie,"Agrumes, Autres",2614,2013,Végétale,0.00,0.00,0.00,0.46,...,1.00,1.00,0.00,0.00,0.00,0.00,0.00,2977000,0.00,"10,866.05"
2,1,Arménie,"Alcool, non Comestible",2659,2013,Végétale,0.00,0.00,0.00,0.00,...,1.00,0.00,0.00,3.00,0.00,0.00,0.00,2977000,0.00,0.00


In [31]:
# Affichage par pays
print(aliments.pivot_table(index='pays', aggfunc='sum')[['dispo_kcal_an','dispo_kgprot_an']].head(3), '\n')

# Affichage par produit
print(aliments.pivot_table(index='produit', aggfunc='sum')[['dispo_kcal_an','dispo_kgprot_an']].head(3))

                       dispo_kcal_an  dispo_kgprot_an
pays                                                 
Afghanistan    23,273,138,760,000.00   649,685,224.80
Afrique du Sud 58,174,984,800,000.00 1,643,732,269.20
Albanie         3,692,166,260,000.00   128,982,608.65 

                               dispo_kcal_an  dispo_kgprot_an
produit                                                      
Abats Comestible       17,239,626,710,000.00 2,799,873,127.20
Agrumes, Autres         2,473,770,345,000.00    52,058,333.05
Alcool, non Comestible                  0.00             0.00


In [32]:
# Test de cohérence : disponibilité de l'Afghanistan
population_afghanistan = int(population[population['pays']=='Afghanistan']['population'])

# Personnes alimentables en calories et protéines avec la dispo. de l'Afghanistan
q3_test_kcal = aliments[aliments['pays']=='Afghanistan']['dispo_kcal_an'].sum() / besoin_kcal
q3_test_kgprot = aliments[aliments['pays']=='Afghanistan']['dispo_kgprot_an'].sum() / besoin_kgprot

print(
    'Avec la disponibilité de l\'Afghanistan, on pourrait alimenter : \n',
    'En calories :', 
    round(q3_test_kcal / 10**6), 'millions de personnes', 
    '(', round(q3_test_kcal / population_afghanistan * 100), '% de la population locale) \n',
    'En protéines :', 
    round(q3_test_kgprot / 10**6), 'millions de personnes', 
    '(', round(q3_test_kgprot / population_afghanistan * 100), '% de la population locale)'
)

Avec la disponibilité de l'Afghanistan, on pourrait alimenter : 
 En calories : 32 millions de personnes ( 104 % de la population locale) 
 En protéines : 32 millions de personnes ( 104 % de la population locale)


## Q4 : Ratio énergie/poids
A partir de ces dernières informations, et à partir du poids de la disponibilité alimentaire (pour chaque pays et chaque produit), calculez pour chaque produit le ratio "énergie/poids", que vous donnerez en kcal/kg. Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet, par exemple en cherchant la valeur calorique d'un oeuf.

In [33]:
# Création des colonnes ratios kcal/kg et pourcentage de protéines
aliments['kcal/kg'] = aliments['dispo_kcal_an'].replace(0, np.nan) / (aliments['nourriture'].replace(0, np.nan) * 1000000)
aliments['% protéines'] = aliments['dispo_kgprot_an'].replace(0, np.nan) / (aliments['nourriture'].replace(0, np.nan) * 1000000) * 100

aliments.dropna(subset=['% protéines'])[aliments['produit']=='Oeufs'][['pays','produit','kcal/kg','% protéines']].sample(3)

<ipython-input-33-6ca98b1f9e72>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  aliments.dropna(subset=['% protéines'])[aliments['produit']=='Oeufs'][['pays','produit','kcal/kg','% protéines']].sample(3)


,pays,produit,kcal/kg,% protéines
7805,République démocratique populaire lao,Oeufs,"1,412.03",10.59
12707,Suriname,Oeufs,"1,377.14",10.53
4733,Allemagne,Oeufs,"1,405.13",11.42


In [34]:
# Test de cohérence : moyennes énergétiques de tous les pays, pour les pommes

# kcal/kg (divisés par 10 pour obtenir la valeur pour 100 grammes)
q4_test_kcal = aliments[aliments['produit']=='Pommes']['kcal/kg'].mean() / 10

# % protéines
q4_test_kgprot = aliments[aliments['produit']=='Pommes']['% protéines'].mean()

print(
    'Moyennes énergétiques mondiales, pour les pommes, comparées aux données de Wikipedia : \n',
    round(q4_test_kcal), 'kcal pour 100g (contre 52 kcal) \n',
    round(q4_test_kgprot, 2), 'grammes de protéines pour 100g (contre 0.26 grammes) \n\n',
    'Source : https://en.wikipedia.org/wiki/Apple'
)

Moyennes énergétiques mondiales, pour les pommes, comparées aux données de Wikipedia : 
 40 kcal pour 100g (contre 52 kcal) 
 0.19 grammes de protéines pour 100g (contre 0.26 grammes) 

 Source : https://en.wikipedia.org/wiki/Apple


## Q5 : Aliments les plus caloriques et protéiques
Citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids.
Citez 5 aliments parmi les 20 aliments les plus riches en protéines.

In [35]:
# 20 aliments les plus caloriques
print(aliments.pivot_table('kcal/kg', index='produit').sort_values(by='kcal/kg', ascending=False).head(20), '\n')

# 20 aliments les plus protéiques
print(aliments.pivot_table('% protéines', index='produit').sort_values(by='% protéines', ascending=False).head(20), '\n')

                          kcal/kg
produit                          
Huiles de Foie de Poisso 9,982.93
Huile de Sésame          9,433.74
Huile de Son de Riz      9,361.55
Huiles de Poissons       9,003.72
Huile d'Arachide         8,935.77
Huile d'Olive            8,931.22
Huile de Germe de Maïs   8,871.76
Huil Plantes Oleif Autr  8,848.22
Huile de Palmistes       8,789.54
Huile Graines de Coton   8,750.32
Huile de Palme           8,719.42
Huile de Tournesol       8,693.25
Huile de Soja            8,635.81
Huile de Colza&Moutarde  8,627.37
Huile de Coco            8,565.74
Graisses Animales Crue   7,583.86
Beurre, Ghee             7,331.42
Palmistes                6,496.64
Sésame                   5,796.12
Arachides Decortiquees   5,702.90 

                          % protéines
produit                              
Graines Colza/Moutarde          28.54
Soja                            28.53
Arachides Decortiquees          25.15
Pois                            22.32
Légumineuses Autres   

In [36]:
# Test de cohérence : moyennes mondiales pour l'huile de sésame et le soja
q5_test_kcal = aliments[aliments['produit']=='Huile de Sésame']['kcal/kg'].mean() / 10 # Résultat pour 100g
q5_test_prot = aliments[aliments['produit']=='Soja']['% protéines'].mean()

print(
    'Moyennes énergétiques mondiales comparées aux données de Wikipedia : \n',
    'Huile de sésame :', round(q5_test_kcal), 'kcal pour 100g (contre 884 kcal) \n',
    'Soja :', round(q5_test_prot, 2), 'grammes de protéines pour 100g (contre 36.49 grammes) \n\n',
    'Sources : \n',
    '- https://en.wikipedia.org/wiki/Sesame_oil \n',
    '- https://en.wikipedia.org/wiki/Soybean \n'
)

Moyennes énergétiques mondiales comparées aux données de Wikipedia : 
 Huile de sésame : 943 kcal pour 100g (contre 884 kcal) 
 Soja : 28.53 grammes de protéines pour 100g (contre 36.49 grammes) 

 Sources : 
 - https://en.wikipedia.org/wiki/Sesame_oil 
 - https://en.wikipedia.org/wiki/Soybean 



## Q6 : Dispo. intérieure mondiale des végétaux (calories)
Calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal.

In [37]:
# Colonnes Disponibilité intérieure en kcal
aliments['dispo_int_kcal'] = aliments['dispo_int'].replace(0, np.nan) * 1000000 * aliments['kcal/kg']

# Disponibilité intérieure mondiale des végétaux (kcal)
dispo_int_kcal_vegetaux = aliments[aliments['origine']=='Végétale']['dispo_int_kcal'].sum()

print(
    'Dispo. intérieure mondiale des végétaux : '
    f'{round(dispo_int_kcal_vegetaux / 10**9):,}', 'milliards de kcal',
)

Dispo. intérieure mondiale des végétaux : 12,312,091 milliards de kcal


In [38]:
# Test de cohérence : 

# Disponibilité intérieure mondiale des produits d'origine animale (kcal)
dispo_int_kcal_animale = aliments[aliments['origine']=='Animale']['dispo_int_kcal'].sum()

# Ratios dispo.interieure mondiale des produits d'origine végétale puis animale / dispo. intérieure mondiale toutes origines
test_q6_vegetaux = round(dispo_int_kcal_vegetaux / aliments['dispo_int_kcal'].sum() * 100)
test_q6_animale = round(dispo_int_kcal_animale / aliments['dispo_int_kcal'].sum() * 100)

print(
    'Proportions de la disponibilité intérieure mondiale pour les produits : \n',
    '- d\'origine végétale :', test_q6_vegetaux, '% \n',
    '- d\'origine animale :', test_q6_animale, '%'
)

Proportions de la disponibilité intérieure mondiale pour les produits : 
 - d'origine végétale : 89 % 
 - d'origine animale : 11 %


## Q7 : Potentiel alimentaire des végétaux
Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [39]:
# Humains qu'on pourrait nourrir avec la dispo. intérieure en végétaux
bouches_kcal_vegetaux = dispo_int_kcal_vegetaux / besoin_kcal

print('En calories :')
print(round(bouches_kcal_vegetaux / 10**9), 'milliards d\'humains')
print(round(bouches_kcal_vegetaux / population_monde * 100), '% de la population mondiale')

En calories :
17 milliards d'humains
241 % de la population mondiale


In [40]:
# Colonnes Disponibilité intérieure en kgprot
aliments['dispo_int_kgprot'] = aliments['dispo_int'].replace(0, np.nan) * 1000000 * aliments['% protéines'] / 100

# Disponibilité intérieure mondiale des végétaux (kgprot)
dispo_int_kgprot_vegetaux = aliments[aliments['origine']=='Végétale']['dispo_int_kgprot'].sum()

# Humains qu'on pourrait nourrir avec la dispo. intérieure en végétaux
bouches_kgprot_vegetaux = dispo_int_kgprot_vegetaux / besoin_kgprot

print('En protéines :')
print(round(bouches_kgprot_vegetaux / 10**9), 'milliards d\'humains')
print(round(bouches_kgprot_vegetaux / population_monde * 100), '% de la population mondiale')

En protéines :
14 milliards d'humains
204 % de la population mondiale


## Q8 : Potentiel alimentaire des végétaux (destinés aux animaux + pertes)
Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux la nourriture végétale destinée aux animaux et les pertes de produits végétaux étaient utilisés pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [41]:
aliments.columns

Index(['code_pays', 'pays', 'produit', 'code_produit', 'annee', 'origine',
       'alim_ani', 'autres_utilisations', 'dispo_alim_kcal_p_j',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'dispo_mat_gr', 'dispo_prot', 'dispo_int', 'Exportations - Quantité',
       'Importations - Quantité', 'nourriture', 'pertes', 'Production',
       'semences', 'transfo', 'Variation de stock', 'population',
       'dispo_kcal_an', 'dispo_kgprot_an', 'kcal/kg', '% protéines',
       'dispo_int_kcal', 'dispo_int_kgprot'],
      dtype='object')

In [42]:
# Valeurs temporaires en kcal
nourriture_kcal = (aliments['nourriture'] * 10**6) * aliments['kcal/kg']
alim_ani_kcal = (aliments['alim_ani'] * 10**6) * aliments['kcal/kg']
pertes_kcal = (aliments['pertes'] * 10**6) * aliments['kcal/kg']

# Valeurs temporaires en kgprot
nourriture_kgprot = (aliments['nourriture'] * 10**6) * (aliments['% protéines'] / 100)
alim_ani_kgprot = (aliments['alim_ani'] * 10**6) * (aliments['% protéines'] / 100)
pertes_kgprot = (aliments['pertes'] * 10**6) * (aliments['% protéines'] / 100)

# Colonnes de sommes des valeurs en kcal et kgprot
aliments['q8_kcal'] = nourriture_kcal + alim_ani_kcal + pertes_kcal
aliments['q8_kgprot'] = nourriture_kgprot + alim_ani_kgprot + pertes_kgprot

# Test de cohérence : ratios q8_kcal et q8_kgprot / Disponibilité intérieure
# → Les ratios sont identiques pour les kcal et les protéines
# → La nourriture + les aliments pour animaux + les pertes représentent souvent la majorité voire la totalité de la disponibilité alimentaire
aliments['q8_test_kcal'] = aliments['q8_kcal'] / ((aliments['dispo_int'] * 10**6) * aliments['kcal/kg'])
aliments['q8_test_kgprot'] = aliments['q8_kgprot'] / ((aliments['dispo_int'] * 10**6) * aliments['% protéines'] / 100)

# Affichage
aliments[[
    'pays',
    'produit',
    'nourriture',
    'alim_ani',
    'pertes',
    'kcal/kg',
    '% protéines',
    'q8_kcal',
    'q8_test_kcal',
    'q8_kgprot',
    'q8_test_kgprot'
]].sample(10)

,pays,produit,nourriture,alim_ani,pertes,kcal/kg,% protéines,q8_kcal,q8_test_kcal,q8_kgprot,q8_test_kgprot
2323,Tchad,Aliments pour enfants,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan
8923,Maurice,Maïs,4.00,95.00,0.00,"3,064.91",7.15,"303,425,595,000.00",0.99,"7,079,930.55",0.99
10961,Philippines,Perciform,417.00,0.00,0.00,430.62,8.10,"179,569,050,000.00",1.00,"33,758,981.40",1.00
15229,Belgique,Raisin,79.00,0.00,5.00,666.94,0.72,"56,023,193,924.05",0.78,"603,326.70",0.78
1149,Bolivie (État plurinational de),Aliments pour enfants,3.00,0.00,0.00,"3,894.91",16.88,"11,684,745,000.00",1.00,"506,338.95",1.00
9926,Vanuatu,Millet,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan
4675,Gambie,"Épices, Autres",0.00,0.00,0.00,nan,nan,nan,nan,nan,nan
12768,Tadjikistan,Huile de Soja,4.00,0.00,0.00,"9,736.74",nan,"38,946,960,000.00",1.00,nan,nan
13861,Turquie,Piments,19.00,0.00,1.00,"2,879.00",11.52,"57,580,094,736.84",1.05,"2,303,203.79",1.05
12262,Sierra Leone,Racines nda,3.00,0.00,0.00,741.19,1.48,"2,223,580,000.00",1.00,"44,471.60",1.00


In [43]:
# Sommes de ces colonnes, pour les végétaux uniquement
q8_sum_kcal = aliments[aliments['origine']=='Végétale']['q8_kcal'].sum()
q8_sum_kgprot = aliments[aliments['origine']=='Végétale']['q8_kgprot'].sum()

# Personnes qu'on pourrait alimenter avec ces sommes
bouches_q8_kcal = q8_sum_kcal / besoin_kcal
bouches_q8_kgprot = q8_sum_kgprot / besoin_kgprot

# Affichage
print(
    'En calories :\n',
    '{:,}'.format(round(q8_sum_kcal / 10**9)), 'milliards de kcal \n',
    '{:,}'.format(round((bouches_q8_kcal / 10**9), 2)), 'milliards d\'humains \n',
    round(bouches_q8_kcal / population_monde * 100), '% de la population mondiale \n\n',
    'En protéines :\n',
    '{:,}'.format(round(q8_sum_kgprot / 10**9)), 'milliards de kgprot \n',
    '{:,}'.format(round((bouches_q8_kgprot / 10**9), 2)), 'milliards d\'humains \n',
    round(bouches_q8_kgprot / population_monde * 100), '% de la population mondiale'
)

En calories :
 9,173,264 milliards de kcal 
 12.57 milliards d'humains 
 180 % de la population mondiale 

 En protéines :
 204 milliards de kgprot 
 10.0 milliards d'humains 
 143 % de la population mondiale


## Q9 : Potentiel alimentaire de la dispo. mondiale
Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [44]:
# Disponibilités alimentaires en kcal et kgprot
aliments['dispo_kcal'] = aliments['nourriture'] * 1000000 * aliments['kcal/kg']
aliments['dispo_kgprot'] = aliments['nourriture'] * 1000000 * aliments['% protéines'] / 100

# Sommes de la disponibilité alimentaire mondiale
dispo_mondiale_kcal = aliments['dispo_kcal'].sum()
dispo_mondiale_kgprot = aliments['dispo_kgprot'].sum()

# Test de cohérence : sommes de la disponibilité intérieure mondiale
dispo_int_mondiale_kcal = aliments['dispo_int_kcal'].sum()
dispo_int_mondiale_kgprot = aliments['dispo_int_kgprot'].sum()

# Calories et protéines potentiellement disponibles pour chaque humain en 2013
potentiel_kcal_personne = dispo_mondiale_kcal / population_monde / 365
potentiel_kgprot_personne = dispo_int_mondiale_kgprot / population_monde / 365 * 1000


# Humains qu'on pourrait nourrir avec cette disponibilité
bouches_dispo_mondiale_kcal = dispo_mondiale_kcal / besoin_kcal
bouches_dispo_mondiale_kgprot = dispo_mondiale_kgprot / besoin_kgprot

print(
    'En calories : \n',
    '{:,}'.format(round(dispo_mondiale_kcal / 10**9)), 'milliards de kcal (',
    round(dispo_mondiale_kcal / dispo_int_mondiale_kcal * 100), '% de la dispo. intérieure mondiale) \n',
    round((bouches_dispo_mondiale_kcal / 10**9), 2), 'milliards d\'humains (',
    round(bouches_dispo_mondiale_kcal / population_monde * 100), '% de la population mondiale) \n',
    'On aurait pu fournir ', round(potentiel_kcal_personne), 'kcal/jour à chaque humain en 2013 \n'
)

print(
    'En protéines : \n',
    '{:,}'.format(round(dispo_mondiale_kgprot / 10**9)), 'milliards de kgprot (',
    round(dispo_mondiale_kgprot / dispo_int_mondiale_kgprot * 100), '% de la dispo. intérieure mondiale) \n',
    round((bouches_dispo_mondiale_kgprot / 10**9), 2), 'milliards d\'humains (',
    round(bouches_dispo_mondiale_kgprot / population_monde * 100), '% de la population mondiale) \n',
    'On aurait pu fournir ', round(potentiel_kgprot_personne), 'grammes de protéines / jour à chaque humain en 2013'
)

En calories : 
 7,360,355 milliards de kcal ( 53 % de la dispo. intérieure mondiale) 
 10.08 milliards d'humains ( 144 % de la population mondiale) 
 On aurait pu fournir  2882 kcal/jour à chaque humain en 2013 

En protéines : 
 207 milliards de kgprot ( 54 % de la dispo. intérieure mondiale) 
 10.17 milliards d'humains ( 145 % de la population mondiale) 
 On aurait pu fournir  149 grammes de protéines / jour à chaque humain en 2013


## Q10 : Pourcentage de la sous-nutrition mondiale
A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

In [45]:
# Restriction sur l'année 2013
sous_nutrition = sous_nutrition[sous_nutrition['annee']=='2012-2014']
sous_nutrition['annee'] = '2013'

sous_nutrition.sample(3)

,code_pays,pays,nb_personnes,annee
570,134,Malte,nan,2013
760,37,République centrafricaine,"2,000,000.00",2013
880,201,Somalie,nan,2013


In [46]:
# Personnes sous-alimentées dans le monde
sousnutrition_monde = sous_nutrition['nb_personnes'].sum()
ratio_sousnutrition = sousnutrition_monde / population_monde

# Pourcentage de la population mondiale sous-alimentée
print(
    'Sous-nutrition :', round(ratio_sousnutrition * 100), '% de la population mondiale \n',
    '(Cohérent avec les 12% pour 2011-2013 estimés par la FAO) \n',
    'Source : http://www.fao.org/3/i3434e/i3434e01.pdf'
)

Sous-nutrition : 11 % de la population mondiale 
 (Cohérent avec les 12% pour 2011-2013 estimés par la FAO) 
 Source : http://www.fao.org/3/i3434e/i3434e01.pdf


## Q11 : Céréales
Établissez la liste des produits (ainsi que leur code) considéré comme des céréales selon la FAO.
En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

In [47]:
# Liste des céréales
liste_cereales = cereales['produit'].drop_duplicates()
liste_cereales

0                  Blé
1     Riz (Eq Blanchi)
2                 Orge
3                 Maïs
4               Millet
9               Seigle
10              Avoine
12              Sorgho
13    Céréales, Autres
Name: produit, dtype: object

In [48]:
# Dataframe n'incluant que les céréales
aliments_cereales = aliments[aliments['produit'].isin(liste_cereales)]

# Pourcentage de céréales destinées à l'alimentation animale
cereales_alim_ani = aliments_cereales['alim_ani'].sum() / (aliments_cereales['nourriture'].sum() + aliments_cereales['alim_ani'].sum()) * 100

# Test de cohérence : pourcentage de tous les aliments d'origine végétale destinés à l'alimentation animale
aliments_vegetaux = aliments[aliments['origine']=='Végétale']
vegetaux_alim_ani = aliments_vegetaux['alim_ani'].sum() / (aliments_vegetaux['nourriture'].sum() + aliments_vegetaux['alim_ani'].sum()) * 100

print(
    'Proportion des céréales destinée à l\'alimentation animale :', 
    round(cereales_alim_ani), '% \n (',
    round(vegetaux_alim_ani), '% si on se base sur l\'ensemble des végétaux)'
)

Proportion des céréales destinée à l'alimentation animale : 46 % 
 ( 24 % si on se base sur l'ensemble des végétaux)


## Q12 : Sous-nutrition
Sélectionnez parmi les données des bilans alimentaires les informations relatives aux pays dans lesquels la FAO recense des personnes en sous-nutrition.

Repérez les 15 produits les plus exportés par ce groupe de pays.

Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné)

Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits. Ensuite, calculez pour chaque produit les 2 quantités suivantes :

le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.
le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

Question 12 : Donnez les 3 produits qui on t la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer)


In [49]:
# Jointure de sous_nutrition et population
sous_nutrition = pd.merge(sous_nutrition, population[['pays','population']], on='pays')

# Colonne pourcentage de sous-nutrition pour chaque pays
sous_nutrition['% sous_nutrition'] = sous_nutrition['nb_personnes'] / sous_nutrition['population'] * 100

sous_nutrition.head(3)

,code_pays,pays,nb_personnes,annee,population,% sous_nutrition
0,2,Afghanistan,"7,900,000.00",2013,30552000,25.86
1,202,Afrique du Sud,"2,600,000.00",2013,52776000,4.93
2,3,Albanie,"200,000.00",2013,3173000,6.30


In [50]:
# Liste des pays en sous-nutrition (+ de 5% de sous-nutrition)
liste_pays_sousnutrition = sous_nutrition[sous_nutrition['% sous_nutrition'] > seuil_sousnutrition].sort_values(by=['% sous_nutrition'], ascending=False)['pays']

print(
    len(liste_pays_sousnutrition), 'pays sous-alimentés (',
    round(len(liste_pays_sousnutrition) / len(sous_nutrition) * 100), '% de la totalité ) \n \n',
    liste_pays_sousnutrition.head(10)
)

83 pays sous-alimentés ( 48 % de la totalité ) 
 
 70                                          Haïti
172                                        Zambie
173                                      Zimbabwe
131                     République centrafricaine
136    République populaire démocratique de Corée
39                                          Congo
157                                         Tchad
5                                          Angola
93                                        Libéria
97                                     Madagascar
Name: pays, dtype: object


In [51]:
# Les 15 produits les plus exportés par les pays en sous-nutrition
pays_sousnutrition_exportes = aliments[aliments['pays'].isin(liste_pays_sousnutrition)].pivot_table('Exportations - Quantité', index='produit').sort_values(by='Exportations - Quantité', ascending=False).head(15).reset_index()['produit']
pays_sousnutrition_exportes

0                  Manioc
1        Riz (Eq Blanchi)
2          Huile de Palme
3                 Bananes
4           Sucre Eq Brut
5                     Blé
6         Légumes, Autres
7                    Maïs
8     Poissons Pelagiques
9          Fruits, Autres
10     Lait - Excl Beurre
11                   Soja
12                 Ananas
13                Tomates
14                   Café
Name: produit, dtype: object

In [52]:
# Les 200 plus grandes importations de ces produits
imports_pays_sousnutrition_exportes = aliments[aliments['produit'].isin(pays_sousnutrition_exportes)].sort_values(by='Importations - Quantité', ascending=False).head(200)
imports_pays_sousnutrition_exportes[['pays','produit','Importations - Quantité']]

,pays,produit,Importations - Quantité
2567,"Chine, continentale",Soja,"63,381.00"
2537,"Chine, continentale",Manioc,"29,046.00"
7015,Japon,Maïs,"14,403.00"
3686,Égypte,Blé,"10,331.00"
7539,République de Corée,Maïs,"8,755.00"
...,...,...,...
5685,Chine - RAS de Hong-Kong,"Fruits, Autres","1,012.00"
10278,Nigéria,Huile de Palme,999.00
3755,Égypte,Sucre Eq Brut,998.00
745,Autriche,Maïs,996.00


In [53]:
# Table pivot
quinze_produits_importes = imports_pays_sousnutrition_exportes.pivot_table(index='produit')

# Ratio autres/dispo.intérieure
quinze_produits_importes['autres_utilisations/dispo_int'] = quinze_produits_importes['autres_utilisations'] / quinze_produits_importes['dispo_int'] * 100

# Ratio nourriture animale / quantité destinée à la nourriture (animale + humaine)
quinze_produits_importes['alim_ani/nourriture_huma_ani'] = quinze_produits_importes['alim_ani'] / (quinze_produits_importes['alim_ani'] + quinze_produits_importes['nourriture']) * 100

quinze_produits_importes

,% protéines,Disponibilité alimentaire en quantité (kg/personne/an),Exportations - Quantité,Importations - Quantité,Production,Variation de stock,alim_ani,annee,autres_utilisations,code_pays,...,pertes,population,q8_kcal,q8_kgprot,q8_test_kcal,q8_test_kgprot,semences,transfo,autres_utilisations/dispo_int,alim_ani/nourriture_huma_ani
produit,,,,,,,,,,,,,,,,,,,,,
Ananas,0.29,6.35,131.00,"2,049.00",171.00,0.00,0.00,2013,0.00,231.00,...,57.00,"320,051,000.00","840,668,605,878.44","6,004,775.76",1.00,1.00,0.00,0.00,0.00,0.00
Bananes,0.74,10.69,440.80,"1,932.60",1.40,0.00,0.00,2013,0.00,195.80,...,68.80,"124,020,000.00","897,578,192,666.66","10,958,116.16",1.00,1.00,0.00,0.00,0.00,0.00
Blé,8.45,92.57,"2,323.50","3,274.39","9,245.22",-148.89,"2,157.93",2013,331.96,137.91,...,352.02,"94,931,978.26","26,564,026,680,368.06","821,534,630.93",0.92,0.92,452.87,130.00,3.30,24.57
Café,5.92,5.51,562.00,"1,493.00",1.50,30.50,0.00,2013,0.00,155.00,...,0.00,"201,389,000.00","412,827,957,500.00","58,622,518.60",1.00,1.00,0.00,0.00,0.00,0.00
"Fruits, Autres",0.58,28.80,814.62,"2,090.75","7,860.50",9.75,0.00,2013,1.50,120.25,...,543.38,"262,655,000.00","3,821,409,472,923.41","47,007,523.69",0.99,0.99,0.00,205.00,0.02,0.00
Huile de Palme,0.45,3.13,"1,907.50","2,647.25","1,700.50",-58.17,0.00,2013,"1,710.67",123.58,...,23.50,"315,389,833.33","7,738,228,206,167.66","2,096,172.28",0.34,0.33,0.00,0.25,71.81,0.00
Lait - Excl Beurre,3.27,155.95,"2,578.66","3,119.45","12,481.28",56.17,"1,051.79",2013,436.52,141.00,...,177.41,"120,423,620.69","6,931,532,386,267.20","413,704,258.40",0.97,0.97,0.00,5.34,3.34,8.42
"Légumes, Autres",1.30,85.38,"1,263.18","2,561.55","6,618.36",1.55,361.82,2013,0.00,142.09,...,661.09,"88,520,818.18","2,078,816,655,796.31","100,399,413.53",1.00,1.00,2.73,14.55,0.00,4.99
Manioc,0.63,12.41,"5,231.40","6,931.00","11,753.00",12.60,"5,029.80",2013,"4,071.00",137.80,...,951.80,"355,007,400.00","16,615,391,214,682.37","115,013,226.34",0.69,0.69,0.00,360.80,30.23,62.24


In [54]:
print('Ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure')
print(quinze_produits_importes['autres_utilisations/dispo_int'].sort_values(ascending=False).head(3), '\n')

print('Ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)')
print(quinze_produits_importes['alim_ani/nourriture_huma_ani'].sort_values(ascending=False).head(3))

Ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure
produit
Huile de Palme   71.81
Manioc           30.23
Maïs             25.92
Name: autres_utilisations/dispo_int, dtype: float64 

Ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)
produit
Maïs                  87.82
Poissons Pelagiques   75.54
Soja                  64.61
Name: alim_ani/nourriture_huma_ani, dtype: float64


## Q13 : USA
Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [55]:
# 10% des céréales destinées aux aliments pour animaux, produites par les USA 
usa_cereales_ani = aliments[(aliments['produit'].isin(liste_cereales)) & (aliments['pays']=='États-Unis d\'Amérique')]['alim_ani'].sum() * .1

# Test de cohérence : total mondial des céréales destinées aux aliments pour animaux
monde_cereales_ani = aliments[aliments['produit'].isin(liste_cereales)]['alim_ani'].sum()
population_usa = int(population[population['pays']=='États-Unis d\'Amérique']['population'])

print(
    '10% des céréales produites par les USA, destinées aux animaux =',
    round(usa_cereales_ani), 'milliers de tonnes de céréales \n',
    '- 100% de ces céréales destinées aux animaux aux USA représentent', round(usa_cereales_ani / monde_cereales_ani * 100 * 10), '% de cette même catégorie dans le monde \n',
    '- sachant que la population étasunienne représente', round((population_usa / population_monde * 100), 1), '% de la population mondiale'
)

10% des céréales produites par les USA, destinées aux animaux = 14010 milliers de tonnes de céréales 
 - 100% de ces céréales destinées aux animaux aux USA représentent 16 % de cette même catégorie dans le monde 
 - sachant que la population étasunienne représente 4.6 % de la population mondiale


In [56]:
# Valeurs nutritionnelles moyennes des céréales
cereales_moyenne_kcal = aliments[aliments['produit'].isin(liste_cereales)]['kcal/kg'].mean()
cereales_moyenne_kgprot = aliments[aliments['produit'].isin(liste_cereales)]['% protéines'].mean()

print('Moyennes des valeurs nutritionnelles des céréales :')
print(round(cereales_moyenne_kcal), 'kcal')
print(round(cereales_moyenne_kgprot), '% \n')

# Personnes qu'on pourrait nourrir avec les 10% de céréales
bouches_cereales_usa_kcal = usa_cereales_ani * 1000000 * cereales_moyenne_kcal / besoin_kcal
bouches_cereales_usa_kgprot = usa_cereales_ani * 1000000 * cereales_moyenne_kgprot /100 / besoin_kgprot

print(
    'Avec les 10% de céréales destinées aux aliments pour animaux des USA, on pourrait potentiellement nourrir : \n\n',
    'En calories :', round(bouches_cereales_usa_kcal / 10**6), 'millions de personnes \n'
    'En protéines :', round(bouches_cereales_usa_kgprot / 10**6), 'millions de personnes \n',
    'Soit en moyenne', round(np.mean([bouches_cereales_usa_kcal,bouches_cereales_usa_kgprot]) / sousnutrition_monde * 100),
    '% de la population mondiale sous-alimentée'
    )

Moyennes des valeurs nutritionnelles des céréales :
2957 kcal
8 % 

Avec les 10% de céréales destinées aux aliments pour animaux des USA, on pourrait potentiellement nourrir : 

 En calories : 57 millions de personnes 
En protéines : 52 millions de personnes 
 Soit en moyenne 7 % de la population mondiale sous-alimentée


## Q14 : Thaïlande
En Thaïlande, quelle proportion de manioc est exportée ? Quelle est la proportion de personnes en sous-nutrition?

In [57]:
# Exportations de manioc en Thaïlande (milliers de tonnes)
manioc_exports_thai = aliments[(aliments['produit']=='Manioc') & (aliments['pays']=='Thaïlande')]['Exportations - Quantité']

# Proportion des exportations de manioc / le total des exportations
ratio_manioc_exports_thai = manioc_exports_thai / aliments[aliments['pays']=='Thaïlande']['Exportations - Quantité'].sum() * 100

# Ratio de personnes sous-alimentées en Thaïlande
ratio_sousnutrition_thai = sous_nutrition[sous_nutrition['pays']=='Thaïlande']['% sous_nutrition']


print(
    'En Thaïlande : \n\n',
    int(manioc_exports_thai), 'milliers de tonnes de manioc exportées \n',
    '(', int(ratio_manioc_exports_thai), '% du total des exportations ) \n\n',
    round(float(ratio_sousnutrition_thai), 2), '% de la population en sous-nutrition'
)

En Thaïlande : 

 25214 milliers de tonnes de manioc exportées 
 ( 49 % du total des exportations ) 

 8.36 % de la population en sous-nutrition


In [58]:
# Valeurs nutritionnelles du manioc en Thaïlande
manioc_thai_kcal = aliments[(aliments['produit']=='Manioc') & (aliments['pays']=='Thaïlande')]['kcal/kg']
manioc_thai_kgprot = aliments[(aliments['produit']=='Manioc') & (aliments['pays']=='Thaïlande')]['% protéines']

bouches_manioc_thai_kcal = int(manioc_exports_thai * 1000000 * manioc_thai_kcal / besoin_kcal)
bouches_manioc_thai_kgprot = int(manioc_exports_thai * 1000000 * manioc_thai_kgprot / 100 / besoin_kgprot)

population_thai = int(population[population['pays']=='Thaïlande']['population'])

print(
    'Valeurs nutritionnelles du manioc en Thaïlande : \n',
    round(int(manioc_thai_kcal)), 'kcal \n',
    round(float(manioc_thai_kgprot), 2), '% de protéines \n\n',
    'Avec les exportations de manioc, on pourrait nourrir : \n'
    'En calories : ', round(bouches_manioc_thai_kcal / 10**6), 'millions de personnes',
    '(', round(bouches_manioc_thai_kcal / population_thai * 100), '% de la population) \n',
    'En protéines : ', round(bouches_manioc_thai_kgprot / 10**6), 'millions de personnes',
    '(', round(bouches_manioc_thai_kgprot / population_thai * 100), '% de la population) \n'
    )

Valeurs nutritionnelles du manioc en Thaïlande : 
 1123 kcal 
 0.39 % de protéines 

 Avec les exportations de manioc, on pourrait nourrir : 
En calories :  39 millions de personnes ( 58 % de la population) 
 En protéines :  5 millions de personnes ( 7 % de la population) 



# **Questions 15 à 20**

## Q15 : Table population
- Clés candidates : 
    - 'code_pays'
    - 'pays'
- Clé primaire retenue : 'pays'

In [59]:
# Tests pour la clé primaire

# Avec la fonction (vérifie toutes les lignes)
pk(population, 'pays')
pk(population, 'code_pays')

# Avec une requête arbitraire
print(
    '\n',
    population[population['pays']=='Albanie'], '\n\n',
    population[population['pays']=='Guatemala'],
)

# Export CSV
population.to_csv('exports/population.csv', index=False)

' pays ' est bien une clé primaire.
' code_pays ' est bien une clé primaire.

    code_pays     pays  population  annee
2          3  Albanie     3173000   2013 

     code_pays       pays  population  annee
67         89  Guatemala    15468000   2013


## Q16 : Table dispo_alim
- Clés candidates :
    - 'pays' + 'produit'
    - 'pays' + 'code_produit'
    - 'code_pays' + 'produit'
    - 'code_pays' + 'code_produit'
- Clé primaire retenue : 'pays' + 'produit'

In [60]:
# Création de la colonne dispo_alim_tonnes (?)
aliments['dispo_alim_tonnes'] = aliments['Disponibilité alimentaire en quantité (kg/personne/an)'] * aliments['population'] * 365 / 1000

dispo_alim = aliments[[
    'pays',
    'code_pays',
    'annee',
    'produit',
    'code_produit',
    'origine',
    'dispo_alim_tonnes', # ?
    'dispo_alim_kcal_p_j',
    'dispo_prot',
    'dispo_mat_gr'
]]

# Tests pour la clé primaire
print(
    dispo_alim[(dispo_alim['pays']=='Soudan') & (dispo_alim['produit']=='Blé')], '\n\n\n',
    dispo_alim[(dispo_alim['pays']=='Japon') & (dispo_alim['produit']=='Café')],
)

# Export CSV
dispo_alim.to_csv('exports/dispo_alim.csv', index=False)

         pays  code_pays  annee produit  code_produit   origine  \
15531  Soudan        276   2013     Blé          2511  Végétale   

       dispo_alim_tonnes  dispo_alim_kcal_p_j  dispo_prot  dispo_mat_gr  
15531     312,056,487.20               197.00        6.18          1.18   


        pays  code_pays  annee produit  code_produit   origine  \
6978  Japon        110   2013    Café          2630  Végétale   

      dispo_alim_tonnes  dispo_alim_kcal_p_j  dispo_prot  dispo_mat_gr  
6978     180,525,408.40                 5.00        0.65          0.00  


## Q17 : Table equilibre_prod
- Clés candidates :
    - 'pays' + 'produit'
    - 'pays' + 'code_produit'
    - 'code_pays' + 'produit'
    - 'code_pays' + 'code_produit'
- Clé primaire retenue : 'pays' + 'produit'

In [61]:
equilibre_prod = aliments[[
    'pays',
    'code_pays',
    'annee',
    'produit',
    'code_produit',
    'dispo_int',
    'alim_ani',
    'semences',
    'pertes',
    'transfo', # ?
    'nourriture',
    'autres_utilisations'
]]

# Tests pour la clé primaire
print(
    equilibre_prod[(equilibre_prod['pays']=='Islande') & (equilibre_prod['produit']=='Vin')],'\n\n\n',
    equilibre_prod[(equilibre_prod['pays']=='Italie') & (equilibre_prod['produit']=='Seigle')],
)

# Export CSV
equilibre_prod.to_csv('exports/equilibre_prod.csv', index=False)

         pays  code_pays  annee produit  code_produit  dispo_int  alim_ani  \
6029  Islande         99   2013     Vin          2655       4.00      0.00   

      semences  pertes  transfo  nourriture  autres_utilisations  
6029      0.00    0.00     0.00        4.00                 0.00   


         pays  code_pays  annee produit  code_produit  dispo_int  alim_ani  \
6667  Italie        106   2013  Seigle          2515      39.00     32.00   

      semences  pertes  transfo  nourriture  autres_utilisations  
6667      1.00    1.00     0.00        6.00                 0.00  


## Q18 : Table sous_nutrition
- Clés candidates : 
    - 'code_pays'
    - 'pays'
- Clé primaire retenue : 'pays'

In [62]:
# Projection
sous_nutrition = sous_nutrition[['pays','code_pays','annee','nb_personnes']]

# Tests pour la clé primaire
print(
    sous_nutrition[sous_nutrition['pays']=='Bulgarie'], '\n\n',
    sous_nutrition[sous_nutrition['pays']=='Colombie'],
)

# Export CSV
sous_nutrition.to_csv('exports/sous_nutrition.csv', index=False)

        pays  code_pays annee  nb_personnes
26  Bulgarie         27  2013    400,000.00 

         pays  code_pays annee  nb_personnes
38  Colombie         44  2013  4,100,000.00


## Q19 : Requêtes SQL

### Pays aux plus hauts ratios
Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, puis en termes de kcal par habitant.

- dispo_prot est exprimé en grammes/habitant/jour
- On divise par 1000 pour obtenir la valeur en kg

```sql
SELECT pays, SUM(dispo_prot/1000) AS sum_prot_kg
FROM dispo_alim
GROUP BY pays
ORDER BY sum_prot_kg DESC
```

En première position, l'Islande obtient 0,133kg de protéines par habitant et par jour, soit largement le double du besoin journalier moyen (environ 55g/habitant/jour)

![title](exports/sum_prot_kg.png)

- dispo_alim_kcal_p_j est déjà exprimé en kcal/habitant/jour

```sql
SELECT pays, SUM(dispo_alim_kcal_p_j) AS sum_kcal
FROM dispo_alim
GROUP BY pays
ORDER BY sum_kcal DESC
```

Les dix premiers pays de cette liste dépassent confortablement le besoin calorique journalier par habitant (environ 2300kcal)

![title](exports/sum_kcal.png)

### Pays aux plus faibles ratios
Pour l'année 2013, les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant.

```sql
SELECT pays, SUM(dispo_prot/1000) AS sum_prot_kg
FROM dispo_alim
GROUP BY pays
ORDER BY sum_prot_kg ASC
```

Avec des disponibilités allant de 37 à 55 grammes, ces dix pays se situent tous au dessous du minimum de 55 grammes de protéines par habitant et par jour. On franchit le seuil de ces 55 grammes dès le 12ème pays de cette liste (Zambie). 

![title](exports/sum_prot_kg_asc.png)

### Volumes des pertes
La quantité totale (en kg) de produits perdus par pays en 2013.

- pertes est exprimé en milliers de tonnes
- On multiplie par 1000000 pour obtenir la valeur en kg

```sql
SELECT pays, SUM(pertes * 1000000) AS sum_pertes
FROM equilibre_prod
GROUP BY pays
```

![title](exports/sum_pertes.png)

Si on classe les pays par volume de produits perdus, on réalise que les dix premiers pays de cette liste appartiennent pour moitié à la liste des pays sous-alimentés.
```sql
SELECT pays, SUM(pertes * 1000000) AS sum_pertes
FROM equilibre_prod
GROUP BY pays
ORDER BY sum_pertes DESC
```

In [63]:
pays_pertes_desc = pd.Series([
    'Chine, continentale',
    'Brésil',
    'Inde',
    'Nigéria',
    'Indonésie',
    'Turquie',
    'Mexique',
    'Égypte',
    'Ghana',
    'États-Unis d\'Amérique'
])

pd.DataFrame([
    pays_pertes_desc,
    pays_pertes_desc.isin(liste_pays_sousnutrition)
])

,0,1,2,3,4,5,6,7,8,9
0,"Chine, continentale",Brésil,Inde,Nigéria,Indonésie,Turquie,Mexique,Égypte,Ghana,États-Unis d'Amérique
1,True,False,True,True,True,False,False,False,True,False


A l'inverse, aucun des 10 pays accusant le moins de pertes ne figure dans la liste des pays sous-alimentés.
```sql
SELECT pays, SUM(pertes * 1000000) AS sum_pertes
FROM equilibre_prod
GROUP BY pays
ORDER BY sum_pertes ASC
```

In [64]:
pays_pertes_asc = pd.Series([
    'Antigua-et-Barbuda',
    'Bermudes',
    'Islande',
    'Kiribati',
    'Maldives',
    'Saint-Kitts-et-Nevis',
    'Chine - RAS de Macao',
    'Bahamas',
    'Barbade',
    'Grenade'
])

pd.DataFrame([
    pays_pertes_asc,
    pays_pertes_asc.isin(liste_pays_sousnutrition)
])

,0,1,2,3,4,5,6,7,8,9
0,Antigua-et-Barbuda,Bermudes,Islande,Kiribati,Maldives,Saint-Kitts-et-Nevis,Chine - RAS de Macao,Bahamas,Barbade,Grenade
1,False,False,False,False,False,False,False,False,False,False


### Proportion de la sous-nutrition
Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.

Requête imbriquée : entre parenthèses, on joint population et sous_nutrition  :
```sql    
SELECT pays, ( nb_personnes / population ) AS ratio
FROM (
    SELECT * FROM population, sous_nutrition
    WHERE population.pays = sous_nutrition.pays
)
ORDER BY ratio DESC
```

Puis :
```sql    
SELECT pays, ( nb_personnes / population ) AS ratio
FROM sous_nutrition_ratio
ORDER BY ratio DESC
```

6 de ces pays apparaissaient déjà dans la liste des pays au plus faible ratio disponibilité alimentaire/habitant en termes de protéines par habitant (Liberia, République centrafricaine, Madagascar, Haïti, Zimbabwe et Congo).

![title](exports/pays_sousnutrition.png)

[Analyse plus appronfondie sur ce résultat ↓](#Table-des-10-pays-les-plus-sous-alimentés)

### Produits au plus haut ratio autres/dispo_int
Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.

```sql    
SELECT produit, ( autres_utilisations / dispo_int ) AS ratio
FROM equilibre_prod
GROUP BY produit
ORDER BY ratio DESC
```

Seuls 6 produits ont un ratio (autres utilisations / dispo. intérieure) supérieur à 0. Tous sont des huiles et graisses, ou des produits dont on extrait l'huile.

![title](exports/autres_dispo.png)

## Q20 : Autres utilisations des produits
Pour quelques uns des produits identifiés dans cette dernière requête SQL, supposez quelles sont ces "autres utilisations" possibles (recherchez sur internet !)

La question précédente a montré que  :
    
       - Huile de palme
       - Arachides décortiquées
       - Huile de plantes oléifères
       - Sorgho
       - Plantes oléifères
       - Graisses animales crues

Les huiles et graisses de ces produits sont très utilisés dans l'alimentation industrielle, les cosmétiques et carburants. 

**Huile de palme** : majoritairement utilisée dans l'élaboration de plats préparés, gâteaux et biscuits, mais aussi pour l'oléochimie (cosmétique, peintures et lubrifiants).

**Arachides décortiquées** : on en extrait de l'huile tranformée en margarine ou utilisée comme huile de table. Cette huile est également utilisée dans la savonnerie et la pharmacopée.

**Huile de plante oléifère** et **plantes oléifères** (colza, tournesol, olivier) : hormis l'utilisation de ces huiles pour l'alimentation humaine, on en produit également des biocarburants.  

**Sorgho** : on en extrait du sucre et des alcools, mais aussi de l'agrocarburant et des biomatériaux.

**Graisses animales crues** : servent de base en savonnerie, cosmétique et même parfumerie

Ils affichent tous une forte valeur calorique ↓

In [65]:
aliments[aliments['produit'].isin(
    [
        'Huile de Palme',
        'Arachides Decortiquees',
        'Huil Plantes Oleif Autr',
        'Sorgho',
        'Plantes Oleiferes, Autre',
        'Graisses Animales Crue'
    ]
)].groupby(['produit']).mean()[['kcal/kg','% protéines']]

,kcal/kg,% protéines
produit,,
Arachides Decortiquees,"5,702.90",25.15
Graisses Animales Crue,"7,583.86",1.45
Huil Plantes Oleif Autr,"8,848.22",1.10
Huile de Palme,"8,719.42",0.49
"Plantes Oleiferes, Autre","4,350.09",17.05
Sorgho,"3,060.42",8.71


# **Analyses supplémentaires**

## Chiffres

### Décès dus à la faim
25000 personnes meurent de faim chaque jour ([Source : FAO](http://www.fao.org/french/newsroom/news/2002/9703-fr.html))

In [66]:
morts_jour = 25000
morts_an = morts_jour * 365
print(
    'Soit', round(morts_an / 10**6), 'millions de morts par an \n (',
    round((morts_an / population_monde * 100), 2),'% de la population mondiale )')

population[(population['population']>9*10**6)&(population['population']<10*10**6)]

Soit 9 millions de morts par an 
 ( 0.13 % de la population mondiale )


,code_pays,pays,population,annee
12,52,Azerbaïdjan,9413000,2013
16,57,Bélarus,9357000,2013
50,225,Émirats arabes unis,9346000,2013
73,97,Hongrie,9955000,2013
148,272,Serbie,9511000,2013
154,210,Suède,9571000,2013


### Nombre de personnes sous-alimentées

In [67]:
pop_usa = int(population[population['pays']=='États-Unis d\'Amérique']['population'])
pop_inde = int(population[population['pays']=='Inde']['population'])

print(
    'En 2013,', round(sousnutrition_monde / 10**6), 'millions de personnes dans le monde étaient en sous-nutrition \n',
    '(', round(ratio_sousnutrition * 100), '% de la population mondiale ) \n',
    '(', round(sousnutrition_monde / pop_usa * 100), '% de la population étasunienne ) \n',
    '(', round(sousnutrition_monde / pop_inde * 100), '% de la population indienne )'
)

En 2013, 744 millions de personnes dans le monde étaient en sous-nutrition 
 ( 11 % de la population mondiale ) 
 ( 232 % de la population étasunienne ) 
 ( 59 % de la population indienne )


### Évolutions possibles pour 2050
Les Nations Unies prévoient que la population mondiale devrait avoisinner 9,7 milliards de personnes en 2050 ([Source : un.org](https://www.un.org/fr/sections/issues-depth/population/index.html#:~:text=Selon%20les%20projections%2C%20la%20population,individus%20vers%20l'an%202100.))

In [68]:
population_monde_2050 = 9.7 * 10**9
ratio_2050_2013 = population_monde_2050 / population_monde
print(
    'Soit', round(ratio_2050_2013 * 100), '% de la population mondiale estimée en 2013.'
)

Soit 139 % de la population mondiale estimée en 2013.


In [69]:
print(
    'Sans évolution positive de la malnutrition, en 2050 :\n', 
    '-', round((population_monde_2050 * ratio_sousnutrition / 10**9), 2),
    'milliard de personnes seront sous-alimentées \n'

    ' -', round(morts_an * ratio_2050_2013 / 10**6),
    'millions de personnes mourront de la faim'

)

Sans évolution positive de la malnutrition, en 2050 :
 - 1.03 milliard de personnes seront sous-alimentées 
 - 13 millions de personnes mourront de la faim


## Causes de la sous-alimentation
Parmi les causes communément admises, on peut citer :
   - Les changements et catastrophes climatiques
   - La problématique de l'eau
   - Les guerres et conflits
   - Les inégalités et la pauvreté
   - La croissance démographique
   - La mauvaise gestion des ressources alimentaires

### Les pertes représentent à elles-seules 5% de la disponibilité intérieure mondiale

In [70]:
dispo_int_mondiale = aliments['dispo_int'].sum()
pertes_mondiales = aliments['pertes'].sum()
nourriture_mondiale = aliments['nourriture'].sum()

ratio_pertes_dispo = pertes_mondiales / dispo_int_mondiale
ratio_nourriture_dispo = nourriture_mondiale / dispo_int_mondiale

print(
    'Les pertes représentent', 
    round(ratio_pertes_dispo * 100),
    '% de la dispo. intérieure mondiale \n',
    'Quand la nourriture destinés aux humains représente', 
    round(ratio_nourriture_dispo * 100),
    '% de la dispo. intérieure mondiale',
)

Les pertes représentent 5 % de la dispo. intérieure mondiale 
 Quand la nourriture destinés aux humains représente 50 % de la dispo. intérieure mondiale


### Table des 10 pays les plus sous-alimentés
La table ci-dessous détaille la relation des [10 pays les plus sous-alimentés au monde](#Proportion-de-la-sous-nutrition) à chacune de ces causes ↓

In [71]:
sous_nutrition_10pays = pd.read_csv('exports/sous_nutrition_10pays.csv')
sous_nutrition_10pays

,pays,catastrophes et changement climatique,accès à l'eau,pauvreté,croissance démographique (2013),conflit (territorial),fin conflit,occupation,libération,régime
0,Haïti,"tempêtes, séismes",faible,"78,00%","1,30%",renversement,1995,étrangère,1934,République
1,Zambie,sécheresse,faible,"87,00%","3,10%",guerre,1988,coloniale,1964,République
2,Zimbabwe,sécheresse,moyen,"74,00%","1,80%",guerre civile,1979,coloniale,1964,République
3,République centrafricaine,inondations,faible,"92,00%","0,30%",guerre civile,2014,coloniale,1960,République
4,République populaire démocratique de Corée,inondations,moyen,?,"0,50%",guerre,actif,étrangère,1948,République
5,Congo,inondations,faible,"82,00%","3,30%",guerre civile,2003,coloniale,1960,République
6,Tchad,"sécheresse, inondations",faible,"86,00%","3,40%",guerre civile,2010,coloniale,1960,République
7,Angola,inondations,faible,"87,00%","3,60%",guerre civile,2002,coloniale,1975,République
8,Libéria,montée des eaux,faible,"92,00%","2,70%",guerre civile,2003,-,1848,République
9,Madagascar,montée des eaux,faible,"97,00%","2,70%",guerre,1945,coloniale,1960,République


On remarque des similitudes quasi-systématiques entre ces 10 pays :

- de très forts taux de pauvreté ([ressources inférieures à 5,50 dollars par personne et par jour](https://en.wikipedia.org/wiki/List_of_countries_by_percentage_of_population_living_in_poverty))
- une croissance démographique forte
- un accès à l'eau potable encore restreint
- une récurrence des épisodes de sécheresse et d'inondations
- un conflit armé ces 30 dernières année
- une domination étrangère jusqu'aux années 60


A titre de comparaison, 2 pays n'appartenant pas à cette liste figurent en queue de liste.

# Sources

- Lien entre eau et nutrition : [actioncontrelafaim.org](https://www.actioncontrelafaim.org/a-la-une/nutrition_eau_world_water_forum/)
- La moitié des personnes sous-alimentées vivent dans des pays touchés par la guerre : [theconversation.com](https://theconversation.com/world-hunger-is-increasing-thanks-to-wars-and-climate-change-84506)
- Le gaspillage représente 1/3 de la nourriture destinée aux humains : [FAO](http://www.fao.org/state-of-food-agriculture/fr/#:~:text=L'estimation%20g%C3%A9n%C3%A9rale%20fournie%20par,perdue%20ou%20gaspill%C3%A9e%20chaque%20ann%C3%A9e.&text=L'Indice%20du%20gaspillage%20alimentaire%2C%20calcul%C3%A9%20par%20le%20PNUE%2C,d%C3%A9tail%20et%20de%20la%20consommation)